In [28]:
# 메모리 변수 모두 제거
all = [var for var in globals() if var[0] != "_"]
for var in all:
    del globals()[var]

# 메모리 변수 확인
#globals()

In [29]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='Malgun Gothic')
from sklearn.linear_model import Lasso #라쏘

from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error  # MSE RMSE
from sklearn.metrics import mean_absolute_error # mae

In [30]:
df1 = pd.read_csv('data/train_메뉴제외.csv')
df3 = pd.read_csv('data/menu_병합.csv')

In [31]:
# 일자 드랍하고 y값 2개 뺴고 전부다 스케일링 해버리자
df1['요일'] =  df1['요일'].astype('category') #요일을 카테고리화 하였다
df1['요일'] = df1.요일.cat.codes #요일을 숫자로 바꿈

y = df1[['요일','중식계', '석식계','일자','중식강수여부','석식강수여부','휴일전날', '휴일다음날']]

x = df1.drop(columns=['요일','중식계', '석식계','일자','중식강수여부','석식강수여부','휴일전날', '휴일다음날']) # 카테고리컬 변수와 y값으로 쓸 변수 일자변수 빼고 전부 스케일링
x = StandardScaler().fit_transform(x)

x = pd.DataFrame(x,columns=df1.drop(columns=['요일','중식계', '석식계','일자','중식강수여부','석식강수여부','휴일전날', '휴일다음날']).columns)
df1 =pd.concat([x,y],axis=1)

In [32]:
df_lunch_cat = df3.iloc[:1205]
df_lunch_cat_dummy =pd.get_dummies(df_lunch_cat.iloc[:1205][['국분류', '메뉴분류']]) # 더미화

df_lunch = pd.concat([df1,df_lunch_cat_dummy],axis=1)

In [33]:
df_dinner_cat_dummy =pd.get_dummies(df3.iloc[1205:][['국분류', '메뉴분류']]) # 더미화
df_dinner_cat_duumy = pd.concat([df3.iloc[1205:]['일자'],df_dinner_cat_dummy],axis=1) # 저녁값이 비어있는곳이 있어 merge 하기 위해 일자 값 추가

df_dinner =pd.merge(df1, df_dinner_cat_duumy,on='일자',how='left')

df_dinner.dropna(inplace=True) #저녁안먹는날 drop

In [34]:
df_co_lunch = df_lunch.iloc[977:]

df_dinner.reset_index(inplace=True)
df_dinner.drop(columns='index',inplace=True)
df_dinner[df_dinner['일자'] == '2020-02-04']

df_co_dinner = df_dinner.iloc[935:] # 코시국 이후 저녁데이터

In [35]:
X3 = df_co_lunch.drop(columns=['일자','중식계','석식계','석식결식','중식결식', '석식기온', '석식강수량', '석식강수여부'])
#X = df_lunch[['요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수',
#       '식사가능인원', '중식강수여부', '중식기온', '중식강수량', '석식강수여부', '석식기온', '석식강수량', '확진자수',
#       '휴일전날', '휴일다음날']]
#X = df[['요일','본사정원수','본사휴가자수','본사출장자수','본사시간외근무명령서승인건수','현본사소속재택근무자수','식사가능인원']] #0.43
y3 = df_co_lunch['중식계']

X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.3, random_state=1)

In [36]:
X4 = df_co_dinner.drop(columns=['일자','중식계','석식계','석식결식','중식결식', '중식기온', '중식강수량', '중식강수여부'])
#X = df[['요일','본사정원수','본사휴가자수','본사출장자수','본사시간외근무명령서승인건수','현본사소속재택근무자수','식사가능인원']] #0.43
y4 = df_co_dinner['석식계']

X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=0.3, random_state=1)

In [37]:
import statsmodels.api as sm
lr = sm.OLS(y3,X3)
result= lr.fit() 
print(result.summary())
print(result.params)
#재택근무자수 무의미함 

                            OLS Regression Results                            
Dep. Variable:                    중식계   R-squared:                       0.835
Model:                            OLS   Adj. R-squared:                  0.820
Method:                 Least Squares   F-statistic:                     55.37
Date:                Fri, 01 Oct 2021   Prob (F-statistic):           1.63e-70
Time:                        11:18:49   Log-Likelihood:                -1351.9
No. Observations:                 228   AIC:                             2744.
Df Residuals:                     208   BIC:                             2812.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
본사정원수            -30.2107     17.043     -1.

In [38]:
import statsmodels.api as sm
lr = sm.OLS(y4,X4)
result= lr.fit() 
print(result.summary())
print(result.params)
#재택근무자수 무의미함 

                            OLS Regression Results                            
Dep. Variable:                    석식계   R-squared:                       0.753
Model:                            OLS   Adj. R-squared:                  0.729
Method:                 Least Squares   F-statistic:                     31.30
Date:                Fri, 01 Oct 2021   Prob (F-statistic):           2.12e-51
Time:                        11:18:49   Log-Likelihood:                -1270.8
No. Observations:                 226   AIC:                             2584.
Df Residuals:                     205   BIC:                             2655.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
본사정원수            -44.7303     12.126     -3.

In [39]:
for alpha in [0.1, 0.5, 1, 10, 100]:
    model_lasso = Lasso(alpha=alpha)
    model_lasso.fit(X3_train, y3_train)
    
    pre =model_lasso.predict(X3_test)
    
    print(f'--------------alpha{alpha}값----------------')
    print(model_lasso.score(X3_test, y3_test)) #R2값 1일수록 좋은것
    print('RMSE=',mean_squared_error(y3_test, pre))
    print('MAE=',mean_absolute_error(y3_test, pre))

--------------alpha0.1값----------------
0.8060605505244636
RMSE= 11227.464479261633
MAE= 79.02246616778511
--------------alpha0.5값----------------
0.8101492971245438
RMSE= 10990.760408268112
MAE= 78.68543017804991
--------------alpha1값----------------
0.815213350714382
RMSE= 10697.594257932324
MAE= 77.74794460057466
--------------alpha10값----------------
0.7993044604299893
RMSE= 11618.58532527584
MAE= 83.6863630187031
--------------alpha100값----------------
0.3323658401480203
RMSE= 38650.40782136135
MAE= 161.27200309879134


In [40]:
for alpha in [0.1, 0.5, 1, 10, 100]:
    model_lasso = Lasso(alpha=alpha)
    model_lasso.fit(X4_train, y4_train)
    
    pre =model_lasso.predict(X4_test)
    
    print(f'--------------alpha{alpha}값----------------')
    print('RMSE=',mean_squared_error(y4_test, pre))
    print('MAE=',mean_absolute_error(y4_test, pre))
    print(model_lasso.score(X4_test, y4_test)) #R2값 1일수록 좋은것

--------------alpha0.1값----------------
RMSE= 5346.036355695127
MAE= 56.909998742839
0.7172251768614746
--------------alpha0.5값----------------
RMSE= 5382.426581954685
MAE= 57.43650136359366
0.7153003415049857
--------------alpha1값----------------
RMSE= 5420.9021577554795
MAE= 57.78815508474421
0.7132652030550513
--------------alpha10값----------------
RMSE= 5806.907472242127
MAE= 59.15574093492146
0.6928477241469229
--------------alpha100값----------------
RMSE= 15336.31079289512
MAE= 102.39426563028252
0.18879665540649204
